# Online Production

In [1]:
#!pip install -U hopsworks --quiet

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()
ms = project.get_model_serving()


Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/550037
2024-05-07 00:26:18,899 WARNING: using legacy validation callback


Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


In [4]:
# Fetch Model from Hopsworks Model Registry
deployment = ms.get_deployment(name="beerratingmodeldeployment")
deployment.start()

Deployment is already running


## 🚀 Send an Inference Request

In [5]:
# Send inference requests to the deployed model as follows:
feature_view = fs.get_feature_view("user_beer_feature_view", 1)

In [6]:
# Initialize serving
feature_view.init_serving(1)    

2024-05-07 00:27:21,145 INFO: Default Online Store Client is set to sql.
2024-05-07 00:27:22,168 INFO: Initialising Vector Server Online SQL client
2024-05-07 00:27:26,285 INFO: Default Online Store Client is set to sql.
2024-05-07 00:27:27,231 INFO: Initialising Vector Server Online SQL client


In [21]:
import datetime

def process_input_vector(beer_beerid, review_time, review_aroma, review_taste, review_appearance, review_palate):
    # Convert review_time to timestamp in milliseconds if not provided
    if review_time is None:
        review_time = int(datetime.datetime.now().timestamp() * 1000)
    
    # Create a dictionary with the input features
    input_data = {
        "instances": [{
            "beer_beerid": beer_beerid,
            "review_time": review_time,
            "review_aroma": review_aroma,
            "review_taste": review_taste,
            "review_appearance": review_appearance,
            "review_palate": review_palate
        }]
    }

    # Return input data as dictionary
    return input_data

# Example input data
beer_beerid = 39971
review_aroma = 6.555556
review_taste = 7.75
review_appearance = 8.666667
review_palate = 6.666667

# Process input vector
input_data = process_input_vector(beer_beerid, None, review_aroma, review_taste, review_appearance, review_palate)

# Get prediction
deployment.predict(input_data)


RestAPIError: Metadata operation error: (url: http://acfdd5a4a839249e8bb85d8b9651a20b-928877002.us-east-2.elb.amazonaws.com/v1/models/beerratingmodeldeployment:predict). Server response: 
HTTP code: 500, HTTP reason: Internal Server Error, body: b'{"error":"HTTPError : HTTP 500: need to call fit or load_model beforehand"}', error code: , error msg: , user msg: 

 Check the model server logs by using `.get_logs()`

In [22]:
# To troubleshoot, you can use the `get_logs()` method
deployment.get_logs()

Explore all the logs and filters in the Kibana logs at https://c.app.hopsworks.ai:443/p/550037/deployments/258051

Instance name: beerratingmodeldeployment-predictor-default-00001-deployme8fqbq
    response, response_headers = await self.dataplane.infer(model_name=model_name, body=body, headers=headers)
  File "/srv/hops/anaconda/envs/theenv/lib/python3.10/site-packages/kserve/protocol/dataplane.py", line 276, in infer
    response = await model(body, headers=headers)
  File "/srv/hops/anaconda/envs/theenv/lib/python3.10/site-packages/kserve/model.py", line 117, in __call__
    else self.predict(payload, headers)
  File "/serving/predictor/../kserve-component-server.py", line 64, in wrapper
    parse_exception(err)
  File "/serving/predictor/../kserve-component-server.py", line 54, in parse_exception
    raise tornado.web.HTTPError(
tornado.web.HTTPError: HTTP 500: need to call fit or load_model beforehand


